## Предисловие
#### В данном докладе будут сравниваться четыре фреймворка для разработки веб приложений. Сначала будет поверхностный гайд по каждому фрейморку, затем будут сравнения с точки зренеия разработки сайта и использования моделей машинного обучения.

## Внедрение модели
Для данного доклада будет использована учебная модель линейной регрессии. Данная модель определяет примерное время поездки такси основываясь на время и день.

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv('./data/train.csv')
df = df[df['trip_seconds'].notna()]
df = df.drop('trip_end_timestamp', axis=1)
df = df.sort_values('trip_start_timestamp')

df['log_trip_seconds'] = np.log1p(df.trip_seconds)

train_df, test_df = df[:10 ** 6].copy(), df[10 ** 6:].copy()

# train_df['log_trip_seconds'] = np.log1p(train_df.trip_seconds)
# test_df['log_trip_seconds'] = np.log1p(test_df.trip_seconds)

train_df.trip_start_timestamp = pd.to_datetime(train_df.trip_start_timestamp)
test_df.trip_start_timestamp = pd.to_datetime(test_df.trip_start_timestamp)

date = train_df.trip_start_timestamp.apply(lambda x: x.date())

In [4]:
def create_features(data_frame):
  X = pd.concat([
    data_frame.trip_start_timestamp.apply(lambda x: x.timetuple().tm_yday),
    data_frame.trip_start_timestamp.apply(lambda x: x.hour)
  ], axis=1, keys=['day', 'hour'])

  return X, data_frame.log_trip_seconds

In [6]:
X_train, y_train = create_features(train_df)
X_test, y_test = create_features(test_df)


train_col = np.array(pd.get_dummies(X_train.hour))
X_train_ohe = np.concatenate((np.array([X_train.day]).T, train_col), axis=1)

test_col = np.array(pd.get_dummies(X_test.hour))
X_test_ohe = np.concatenate((np.array([X_test.day]).T, test_col), axis=1)

In [7]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_ohe, y_train)

LinearRegression()

In [8]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, lin_reg.predict(X_test_ohe))

4.433441738913538

#### После чего мы сохраняем модель в помощью pickle. Осталось лишь открыть полученный файл в любом фрейморке.

In [9]:
import pickle

path = './data/models/'
with open(path + 'lin_reg.pkl', 'wb') as file:
    pickle.dump(lin_reg, file)

## Flask
Разработка на фласк сводится к написанию функций с декораторами, которые описывают маршрут пользователя. Для того чтобы определить наше приложение, нужно импортировать из библиотеки flask класс Flask и создать экземпляр, передав аргументом имя текущего файла 
$ \\ $ app = Flask(__ name__)  $ \\ $
Теперь можно запустить локальный сервер с помощью метода run(). 
#### Декоратор @app.route('/')
Данный декоратор нужен нужен для маршрутизации, в аргументе хранится маршурт страницы. Таким образом, можно создать функцию index(), которая будет возвращать html страницу.
$ \\ $
С помощью методов GET и POST будет реализована логика обмена информацией между пользователем и моделью. Следует отметить простоту данного фреймворка, для маленьких проектов он очень удобен, так как весь проект нужно реализовывать с нуля.

## FastAPI
С развитием технологий появился молниеносный ASGI-сервер(Asynchronous Server Gateway Interface) под названием Uvicorn. Однако Uvicorn  —  всего лишь веб-сервер без каких-либо возможностей маршрутизации. Затем появился Starlette, который предоставляет полный набор инструментов ASGI поверх ASGI-сервера, такого как Uvicorn, Daphne или Hypercorn. Таким образом, Starlette  — это веб-фреймворк ASGI, а Flask  — веб-фреймворк WSGI.

Фреймворк FastAPI в полной мере использует функциональные возможности Starlette и стиль программирования Flask. Процесс создания веб-приложения очень схож с процессом во Flask, также нужно вешать на функции декораторы, также проект создается с нуля. Есть небольшие отличия в синтаксисе, на пример вместо метода route используются сразу методы get, post, put, delete. Можно использовать тот же шаблонизатор jinja.

Для того чтобы запустить сервер, нужно прописать команду "uvicorn main:app", где main - имя исполняемого файла, а app - экземпляр класса FastAPI

## Django
Высокоуровневый Python веб-фреймворк, который позволяет быстро создавать безопасные и поддерживаемые веб-сайты. Созданный опытными разработчиками, Django берёт на себя большую часть хлопот веб-разработки, что дает программисту сосредоточиться на написании своего веб-приложения без необходимости изобретать велосипед. Проект написанный на Django состоит из "приложений", которые создаются с помощью команды. Преимущество такого подхода к разработке состоит в том, что разработчик может переносить целое приложение из одного проекта в другой с минимальной настройкой подключения приложения к проекту. Django, это фреймворк, в котором есть почти все решения, а если их нет, можно написать свое.

Некоторые команды:
- Создание проекта- "django-admin startproject project_name"
- Запуск сервера- "python manage.py runserver". 
- Создание приложения- "python manage.py startapp app_name"

## gRPC
Микросервисная архитектура – способ организации сложных программных систем: приложение разбивается на сервисы, которые развертываются независимо, но взаимодействуют друг с другом. Одно из главных преимуществ gRPC это proto файлы, которые генерируют api на любом языке. Такой подход позволяет обобщить разработку веб-приложения, что сэкономит время.

- Установка gRPC - "pip install grpcio"
- Установка gRPC tools для генерации кода клиента и сервера - "python pip install grpcio-tools"
- Генерация api - "python -m grpc_tools.protoc -I ../protobufs --python_out=. --grpc_python_out=. ../protobufs/datetime.proto"

## Сравнение
- Flask очень простой фреймворк, он почти пустой в сравнении с остальными фрейморками. Эта особенность может быть как преимуществом, так и недостатком, можно очень быстро сделать веб-приложение, однако на серьезном большом проекте придется хорошо подумать над тем как организовать логику веб-приложения.


- Django монструозен, в нем есть почти все, что нужно в стандартных ситуациях, а если нет, то можно самому написать. Такой фреймворк очень хорошо подходит для создания больших проектов, не в последнюю очередь из-за приложений Django.


- FastAPI лично из моего опыта, он как Flask, только лучше. Есть поддержка асинхронного программирования, автоматическая документация, высокая скорость из-за Scarlette. Синтаксис очень похожи на тот, что был в Flask.


- gRPC до сих пор все выше перечисленные фреймворки представляли собой REST API, где клиент посылает серверу http запрос, в котором может предать json-файл, а сервер, в свою очередь, делает http ответ, в котором также может передать json. gRPC - Remote Procedure Call (g - в каждой версии данного фреймворка означало что-то свое), это дает нам вызывать на клиентской стороне методы сервера, хотя в файле клиента эти методы не были прописаны. Преимущество такого метода заключается в использовании вместо json-файлов protobuf-файлы, которые гораздо меньше весят. Также не стоит забывать о генерации api на любом языке.

## Репозитории
- Flask: https://github.com/drogovozDP/Flask_ML
- FastAPI: https://github.com/drogovozDP/FastAPI_ML
- Django: https://github.com/drogovozDP/Django_ML
- gRPC: https://github.com/drogovozDP/gRPC_ML